# Это ноутбук для построения дашбордов по базам данным для НИР "Алгоритмическое ценообразование"

In [81]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

## Каршеринг

### График числа значений для каждого маршрута на каждый день

Для ситидрайва
```postgresql
SELECT
    DATE(created_at) AS day,
    route_id,
    COUNT(*) AS count
FROM city_drive
GROUP BY day, route_id
ORDER BY day, route_id
```
Для делимобиля
```postgresql
SELECT
    DATE(created_at) AS day,
    route,
    COUNT(*) AS count
FROM deli_mobile
GROUP BY day, route
ORDER BY day, route
```

In [82]:
def make_monitoring_plot(path: str = "data/cars/мониторинг дели.csv"):
    """
    Создает график числа значений для каждого маршрута по дням, заполняя пропуски нулями.
    Данные в этом графике используются для мониторинга парсера в рантайме
    Args:
        path (str): Путь к CSV файлу с данными. Файл должен содержать столбцы "day", "route_id" и "count".
    """
    df = pd.read_csv(path)
    df['route_id'] = df['route_id'].astype(str)
    df['day'] = pd.to_datetime(df['day'])

    min_date = df['day'].min()
    max_date = df['day'].max()

    all_dates = pd.date_range(min_date, max_date)

    all_data = pd.DataFrame([(date, route) for date in all_dates for route in df['route_id'].unique()],
                            columns=['day', 'route_id'])

    df = pd.merge(all_data, df, on=['day', 'route_id'], how='left').fillna(0)
    df['count'] = df['count'].astype(int) # Преобразуем count в int после заполнения NaN

    # Строим график
    fig = px.bar(df, x="day", y="count", color="route_id",
                 title="Число значений для каждого маршрута по дням",
                 labels={"count": "Число значений", "day": "День", "route_id": "Маршрут"})
    fig.update_layout(
        xaxis_title="День",
        yaxis_title="Число значений",
        legend_title="Маршрут",
        xaxis={'categoryorder':'category ascending'}
    )
    fig.show()


In [83]:
make_monitoring_plot("data/cars/мониторинг дели.csv")

In [84]:
make_monitoring_plot("data/cars/мониторинг сити.csv")

### График средней цены за день

Для ситидрайва
```postgresql
SELECT
    DATE(created_at) AS date,
    AVG(price) AS avg_price,
    AVG(fix) AS avg_fix,
    AVG(price_per_minute) AS avg_price_per_minute
FROM
    city_drive
GROUP BY
    DATE(created_at)
ORDER BY
    DATE(created_at);
```

Для делимобиля
```postgresql
SELECT
    DATE(dm.created_at) AS day,
    AVG(dt30.total) AS avg_total_hours_2_km_30,
    AVG(dt15.total) AS avg_total_hours_2_km_15,
    AVG(dm.fix) AS avg_fix,
    AVG(dm.price_per_minute) AS avg_price_per_minute,
    AVG(dm.start) AS avg_start
FROM
    deli_mobile dm
JOIN
    deli_tariff dt30 ON dm.hours_2_km_30 = dt30.id
JOIN
    deli_tariff dt15 ON dm.hours_2_km_15 = dt15.id
GROUP BY
    day
ORDER BY
    day;
```

In [85]:
import pandas as pd

def make_monitoring_plot_deli(path: str = 'data/cars/средние цены на тарифы в день дели.csv'):
    """
    График средних цен у Делимобиля
    Принимает путь до файла с данными
    Необходимые поля:
    "day","avg_total_hours_2_km_30",
    "avg_total_hours_2_km_15","avg_fix",
    "avg_price_per_minute","avg_start"
    """
    df = pd.read_csv(path)
    df['day'] = pd.to_datetime(df['day'])
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['day'],
        y=df['avg_total_hours_2_km_30'],
        mode='lines+markers',
        name='Avg Total Hours (2km, 30 min)')
    )
    fig.add_trace(go.Scatter(
        x=df['day'],
        y=df['avg_total_hours_2_km_15'],
        mode='lines+markers',
        name='Avg Total Hours (2km, 15 min)')
    )
    fig.add_trace(go.Scatter(
        x=df['day'],
        y=df['avg_fix'],
        mode='lines+markers',
        name='Avg Fix')
    )
    fig.add_trace(go.Scatter(
        x=df['day'],
        y=100 * df['avg_price_per_minute'],
        mode='lines+markers',
        name='100 * Avg Price per Minute')
    )

    fig.update_layout(
        title='Изменение показателей во времени',
        xaxis_title='Дата',
        yaxis_title='Значение',
        template='plotly_white',
        hovermode='x unified'
    )

    fig.show()

In [113]:
make_monitoring_plot_deli('data/cars/средние цены на тарифы в день дели.csv')

In [87]:
def make_monitoring_plot_city(path: str = "data/cars/средние цены на тарифы за день сити.csv"):
    """
    График средних цен у СитиДрайва
    Принимает путь до файла с данными
    Необходимые поля: "date","avg_price","avg_fix","avg_price_per_minute"
    """
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'])
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df['avg_price'],
        mode='lines+markers',
        name='Avg Price')
    )
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df['avg_fix'],
        mode='lines+markers',
        name='Avg Fix')
    )
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=50*df['avg_price_per_minute'],
        mode='lines+markers',
        name='50 * Avg Price per Minute')
    )

    fig.update_layout(
        title='Изменение показателей во времени',
        xaxis_title='Дата',
        yaxis_title='Значение',
        template='plotly_white',
        hovermode='x unified'
    )
    fig.show()

In [88]:
make_monitoring_plot_city("data/cars/средние цены на тарифы за день сити.csv")

### График числа значений в данных

Для ситидрайва:

```postgresql
SELECT DATE(created_at) AS day, COUNT(*) AS records_count
FROM (
    -- SELECT created_at FROM deli_mobile
    -- UNION ALL
    SELECT created_at FROM city_drive
) AS combined_data
GROUP BY day
ORDER BY day;
```

Для делимобиля

```postgresql
SELECT DATE(created_at) AS day, COUNT(*) AS records_count
FROM (
    SELECT created_at FROM deli_mobile
    -- UNION ALL
    -- SELECT created_at FROM city_drive
) AS combined_data
GROUP BY day
ORDER BY day;
```

In [89]:
def count_of_records(path: str = 'data/cars/число записей по дням дели.csv'):
    """
    График количества записей по дням
    Необходимые переменные: "day","records_count"
    """
    df = pd.read_csv(path)
    df['day'] = pd.to_datetime(df['day'])
    start_date = pd.to_datetime('2025-03-01')
    end_date = datetime.now()
    date_range = pd.date_range(start=start_date, end=end_date)
    date_range_df = pd.DataFrame({'day': date_range})

    df = pd.merge(date_range_df, df, on='day', how='left')
    df['records_count'] = df['records_count'].fillna(0)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df['day'], y=df['records_count'], name='Records Count'))

    fig.update_layout(
        title='Количество записей по дням',
        xaxis_title='Дата',
        yaxis_title='Количество записей',
        template='plotly_white',
        xaxis_tickformat='%Y-%m-%d'
    )
    fig.show()


In [90]:
count_of_records('data/cars/число записей по дням дели.csv')

In [91]:
count_of_records('data/cars/число записей по дням сити.csv')

### График числа значений для каждого маршрута

Для ситидрайва

```postgresql
SELECT route_id, COUNT(*) AS records_count
FROM (
    --SELECT route AS route_id FROM deli_mobile
    --UNION ALL
    SELECT route_id FROM city_drive
) AS routes
GROUP BY route_id
ORDER BY route_id;
```

Для делимобиля
```postgresql
SELECT route_id, COUNT(*) AS records_count
FROM (
    SELECT route AS route_id FROM deli_mobile
    -- UNION ALL
    -- SELECT route_id FROM city_drive
) AS routes
GROUP BY route_id
ORDER BY route_id;

In [92]:
import pandas as pd
import plotly.graph_objects as go

def make_count_of_records_per_route(path: str = "data/cars/число значений для маршрута дели.csv"):
    """
    График количества записей в данных для каждого маршрута
    Необходимые переменные: "route_id","records_count"
    """
    df = pd.read_csv(path)
    df['route_id'] = df['route_id'].astype(str)

    fig = go.Figure()

    fig.add_trace(go.Bar(x=df['route_id'], y=df['records_count'], name='Records Count'))

    fig.update_layout(
        title='Количество записей по номерам маршрутов',
        xaxis_title='Номер маршрута',
        yaxis_title='Количество записей',
        template='plotly_white'
    )
    fig.show()

In [93]:
make_count_of_records_per_route("data/cars/число значений для маршрута дели.csv")

In [94]:
make_count_of_records("data/cars/число значений для маршрута сити.csv")

### График изменения цены для каждого маршрута

Для ситидрайва

```postgresql
SELECT
    DATE(cd.created_at) AS date,
    cd.route_id AS route_id,
    AVG(cd.price) AS avg_price,
    AVG(cd.fix) AS avg_fix,
    AVG(cd.price_per_minute) AS avg_price_per_minute
FROM
    city_drive cd
GROUP BY
    DATE(cd.created_at),
    cd.route_id
ORDER BY
    DATE(cd.created_at),
    cd.route_id;
```

Для делимобиля
```postgresql
SELECT
    DATE(dm.created_at) AS date,
    dm.route AS route_id,
    AVG(dt30.total) AS avg_total_2km_30,
    AVG(dt15.total) AS avg_total_2km_15,
    AVG(dm.fix) AS avg_fix,
    AVG(dm.price_per_minute) AS avg_price_per_minute,
    AVG(dm.start) AS avg_start
FROM
    deli_mobile dm
LEFT JOIN
    deli_tariff dt30 ON dm.hours_2_km_30 = dt30.id
LEFT JOIN
    deli_tariff dt15 ON dm.hours_2_km_15 = dt15.id
GROUP BY
    DATE(dm.created_at),
    dm.route
ORDER BY
    DATE(dm.created_at),
    dm.route;
```

In [95]:
import pandas as pd
import plotly.graph_objects as go

def make_plot_tariffs_different_routes(variable: str, service: str = 'Сити'):
    """
    Для введенной variable датасета выдает график ее средних значений на каждом из маршрутов
    Необходимые переменные:
    "date","route_id","avg_total_2km_30",
    "avg_total_2km_15","avg_fix",
    "avg_price_per_minute","avg_start"
    """
    path: str = f'data/cars/средние цены на тарифы по маршрутам {service.lower()}.csv'
    df = pd.read_csv(path)
    df['date'] = pd.to_datetime(df['date'])
    fig = go.Figure()

    for route_id in sorted(df['route_id'].unique()):
        route_df = df[df['route_id'] == route_id]
        fig.add_trace(go.Scatter(
            x=route_df['date'],
            y=route_df[variable],
            mode='lines+markers',
            name=f'Route {route_id}',
            hoverinfo='x+y+name',
            line=dict(width=2)
        ))

    fig.update_layout(
        title=dict(
            text=f'Изменение {variable} по маршрутам во времени в сервисе {service}',
            x=0.5,
            y=0.95,
            xanchor='center',
            font=dict(size=16)
        ),
        margin=dict(t=120),
        xaxis_title='Дата',
        yaxis_title=variable,
        template='plotly_white',
        hovermode='x unified',
        showlegend=True,
        legend=dict(
            title='Маршруты',
            orientation='h',
            yanchor='top',
            y=-0.25,
            xanchor='center',
            x=0.5
        )
    )
    fig.show()

Сначала для сити драйва построим все графики

In [96]:
make_plot_tariffs_different_routes('avg_fix')

In [97]:
make_plot_tariffs_different_routes('avg_price_per_minute')

In [98]:
make_plot_tariffs_different_routes('avg_price')

Теперь для делимобиля

In [99]:
make_plot_tariffs_different_routes('avg_fix', "Дели")

In [100]:
make_plot_tariffs_different_routes('avg_price_per_minute', "Дели")

In [101]:
make_plot_tariffs_different_routes('avg_total_2km_30', "Дели")

In [102]:
make_plot_tariffs_different_routes('avg_start', "Дели")

## Авиабилеты

### График средней цены за день

```sql
SELECT
    flight_date AS day,
    AVG(lowest_price) AS avg_price
FROM public.regular_airport_monitoring_data
WHERE
    EXTRACT(YEAR FROM flight_date) = 2025
GROUP BY flight_date
ORDER BY flight_date;
```

In [103]:
import pandas as pd
import plotly.express as px

def make_avg_price_plot(path: str = "data/air/средние-цены-на-дату.csv"):
    """
    График средней цены на рейс
    Необходимые переменные: "flight_date","avg_price"
    """
    df = pd.read_csv(path)
    df['flight_date'] = pd.to_datetime(df['flight_date'])
    df_2025 = df[df['flight_date'].dt.year >= 2025]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df_2025['flight_date'],
        y=df_2025['avg_price'],
        mode='lines+markers',
        name='Average Flight Price in 2025')
    )
    fig.update_layout(
         title=dict(
            text="Средняя цена на рейс",
            x=0.5,
            font=dict(size=16)
        ),
        xaxis_title='Date',
        yaxis_title='Average Price',
        template='plotly_white'
    )
    fig.update_traces(
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Average Price: %{y:.2f}<extra></extra>'
    )
    fig.show()

In [104]:
make_avg_price_plot()

### Топ-10 популярных направлений (пар аэропортов)

```postgresql
WITH airport_pairs AS (
    SELECT
        rap.airport1_id,
        rap.airport2_id,
        a1.code AS airport1_code,
        a2.code AS airport2_code,
        COUNT(f.id) AS flights_count
    FROM public.regular_airports_pairs rap
    LEFT JOIN public.flight f
        ON (f.airport_origin_id = rap.airport1_id AND f.airport_destination_id = rap.airport2_id)
        OR (f.airport_origin_id = rap.airport2_id AND f.airport_destination_id = rap.airport1_id)
    LEFT JOIN public.airport a1 ON rap.airport1_id = a1.id
    LEFT JOIN public.airport a2 ON rap.airport2_id = a2.id
    WHERE
        EXTRACT(YEAR FROM f.local_departure_date_time) = 2025
    GROUP BY rap.airport1_id, rap.airport2_id, a1.code, a2.code
)
SELECT
    CONCAT(airport1_code, ' — ', airport2_code) AS airport_pair,
    flights_count
FROM airport_pairs
ORDER BY flights_count DESC
LIMIT 10;
```

In [105]:
import pandas as pd
import plotly.express as px

def make_plot_top_10_pairs(path: str = "data/air/самые-популярные-аэропорты.csv"):
    """
    Визуализация топ-10 самых популярных пар аэропортов
    Необходимые переменные: "airport_pair","flights_count"
    """
    df = pd.read_csv(path)
    df['flights_count'] = df['flights_count'].astype(int)
    df_grouped = df.groupby('airport_pair')['flights_count'].sum().reset_index()
    df_sorted = df_grouped.sort_values('flights_count', ascending=False)
    df_top10 = df_sorted.head(10)
    fig = px.bar(df_top10,
                 x='airport_pair',
                 y='flights_count',
                 color='flights_count',
                 color_continuous_scale=px.colors.sequential.Viridis,
                 title='Топ-10 популярных направлений (пар аэропортов)',
                 labels={'airport_pair': 'Пара аэропортов', 'flights_count': 'Количество рейсов'})
    fig.update_layout(
        xaxis_title='Пара аэропортов',
        yaxis_title='Количество рейсов',
        title_x=0.5,
        title_font_size=20,
        xaxis={'categoryorder':'total descending'},
        template="plotly_white"
    )

    fig.show()


In [106]:
make_plot_top_10_pairs()

### Таблица динамики цен на авиабилеты в зависимости от времени между поиском билета и датой вылета

sql запрос, с помощью которого достаются данные:

```postgresql
WITH booking_periods AS (
    SELECT
        rmd.flight_date,
        rmd.lowest_price,
        ((rmd.flight_date - ts.search_date) / 7) AS weeks_before
    FROM regular_flight_monitoring_data rmd
    JOIN tasks_stats ts ON rmd.regular_flight_id = ts.regular_flight_id
    WHERE (rmd.flight_date - ts.search_date) BETWEEN 0 AND 84

    UNION ALL

    SELECT
        amd.flight_date,
        amd.lowest_price,
        ((amd.flight_date - ats.search_date) / 7) AS weeks_before
    FROM regular_airport_monitoring_data amd
    JOIN airport_tasks_stats ats ON amd.regular_airports_id = ats.id
    WHERE (amd.flight_date - ats.search_date) BETWEEN 0 AND 84
)

SELECT
    weeks_before,
    COUNT(*) AS observations_count,
    AVG(lowest_price)::INTEGER AS avg_price,
    MIN(lowest_price) AS min_price,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY lowest_price)::INTEGER AS price_25pctl,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY lowest_price)::INTEGER AS median_price,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY lowest_price)::INTEGER AS price_75pctl,
    MAX(lowest_price) AS max_price
FROM booking_periods
WHERE weeks_before IN (0, 3, 6, 9, 12)
GROUP BY weeks_before
ORDER BY weeks_before DESC;
```

Variant 2 with only 2025 year

```sql
WITH flight_stats AS (
    SELECT
        -- Рассчитываем разницу в неделях между датой вылета и датой создания записи (как приближение для search_date)
        FLOOR(DATE_PART('day', flight_date - created_at) / 7)::INT AS weeks_before,
        lowest_price
    FROM public.regular_airport_monitoring_data
    WHERE
        EXTRACT(YEAR FROM flight_date) = 2025
        AND DATE_PART('day', flight_date - created_at) >= 0  -- Исключаем отрицательные значения (поиск после вылета)
        AND FLOOR(DATE_PART('day', flight_date - created_at) / 7) IN (0, 3, 6, 9, 12)
)
SELECT
    weeks_before,
    COUNT(*) AS observations_count,
    ROUND(AVG(lowest_price)) AS avg_price,
    MIN(lowest_price) AS min_price,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY lowest_price)::INT AS price_25pctl,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY lowest_price)::INT AS median_price,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY lowest_price)::INT AS price_75pctl,
    MAX(lowest_price) AS max_price
FROM flight_stats
GROUP BY weeks_before
ORDER BY weeks_before;

-- second query

WITH flight_stats AS (
    SELECT
        -- Используем CEIL вместо FLOOR для корректного округления вверх
        CEIL(DATE_PART('day', flight_date - created_at) / 7)::INT AS weeks_before,
        lowest_price
    FROM public.regular_airport_monitoring_data
    WHERE
        EXTRACT(YEAR FROM flight_date) = 2025
        AND flight_date > created_at -- Исключаем случаи, когда дата вылета раньше поиска
        AND CEIL(DATE_PART('day', flight_date - created_at) / 7) IN (0, 3, 6, 9, 12)
)
SELECT
    weeks_before,
    COUNT(*) AS observations_count,
    ROUND(AVG(lowest_price)) AS avg_price,
    MIN(lowest_price) AS min_price,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY lowest_price)::INT AS price_25pctl,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY lowest_price)::INT AS median_price,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY lowest_price)::INT AS price_75pctl,
    MAX(lowest_price) AS max_price
FROM flight_stats
GROUP BY weeks_before
ORDER BY weeks_before;
```

In [107]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def make_price_dynamic_plot(path: str = 'data/air/динаминка-цен.csv'):
    """
    График изменения метрик цены на разных интервалах от текущей даты: 0,3,6,9,12
    Необходимые переменные:
    "weeks_before","observations_count",
    "avg_price","min_price","price_25pctl",
    "median_price","price_75pctl","max_price"
    """
    df = pd.read_csv(path)

    fig = make_subplots(
        rows=7, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(
            "Средняя цена",
            "Минимальная цена",
            "25-й перцентиль",
            "Медианная цена",
            "75-й перцентиль",
            "Максимальная цена",
            "Количество наблюдений"
        )
    )

    price_metrics = [
        ('avg_price', '#1f77b4'),
        ('min_price', '#ff7f0e'),
        ('price_25pctl', '#2ca02c'),
        ('median_price', '#d62728'),
        ('price_75pctl', '#9467bd'),
        ('max_price', '#8c564b')
    ]

    for row, (col, color) in enumerate(price_metrics, 1):
        fig.add_trace(go.Scatter(
            x=df['weeks_before'],
            y=df[col],
            mode='lines+markers',
            name=col.replace('_', ' ').title(),
            line=dict(color=color, width=2),
            marker=dict(size=8, color=color),
            hoverinfo='x+y+name'
        ), row=row, col=1)

    fig.add_trace(go.Bar(
        x=df['weeks_before'],
        y=df['observations_count'],
        name='Количество наблюдений',
        marker=dict(color='#2ca02c', opacity=0.7),
        hoverinfo='y+name'
    ), row=7, col=1)

    fig.update_xaxes(
        tickmode='array',
        tickvals=df['weeks_before'],
        ticktext=[str(x) for x in df['weeks_before']],
        row=7, col=1
    )

    fig.update_layout(
        title_text='Анализ ценовой динамики по неделям',
        height=1400,
        showlegend=False,
        template='plotly_white',
        hovermode='x unified'
    )

    for i in range(1, 7):
        fig.update_yaxes(title_text="Цена", row=i, col=1)

    fig.update_yaxes(title_text="Количество", row=7, col=1)
    fig.update_xaxes(title_text="Недель до события", row=7, col=1)

    fig.show()

In [108]:
make_price_dynamic_plot()

### График количества собранных авиабилетов в день

```postgresql
SELECT
    DATE(f.created_at) AS day,
    COUNT(f.id) AS flights_count
FROM public.flight f
WHERE
    EXTRACT(YEAR FROM f.created_at) = 2025
GROUP BY day
ORDER BY day;
```

In [109]:
import pandas as pd
import plotly.graph_objects as go


def make_daily_count_plot(path: str = "data/air/количество-собранных-рейсов-в-день.csv"):
    """
    График количества собранных данных о рейсах в день (просто операции вставки)
    Необходимые переменные: "day","flights_count"
    """
    df = pd.read_csv(path)
    df['day'] = pd.to_datetime(df['day'])
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['day'],
        y=df['flights_count'],
        mode='lines+markers',
        name='Количество собранных рейсов')
    )
    fig.update_layout(
         title=dict(
            text="Количество собранных данных в день",
            x=0.5,
            font=dict(size=16)
        ),
        xaxis_title='Дата',
        yaxis_title='Количество собранных рейсов в день',
        template='plotly_white'
    )
    fig.update_traces(
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Количество: %{y:.2f}<extra></extra>'
    )
    fig.show()

In [110]:
make_daily_count_plot()

### График числа собранных билетов на конкретную дату

```sql

SELECT flight_date, COUNT(*) as records_count
FROM (
    SELECT flight_date FROM regular_flight_monitoring_data
    UNION ALL
    SELECT flight_date FROM regular_airport_monitoring_data
) AS combined_data
GROUP BY flight_date
ORDER BY flight_date;
```

In [111]:
def make_flights_count_plot(path: str = "data/air/количество-билетов-на-дату.csv"):
    """
    График числа собранных билетов на конкретную дату
    Необходимые переменные: "flight_date","records_count"
    """
    df = pd.read_csv(path)
    df['flight_date'] = pd.to_datetime(df['flight_date'])
    df = df[df['flight_date'].dt.year >= 2025]
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['flight_date'],
        y=df['records_count'],
        mode='lines+markers',
        name='Количество рейсов на дату')
    )
    fig.update_layout(
        title=dict(
            text="Количество рейсов по датам",
            x=0.5,
            font=dict(size=16)
        ),
        xaxis_title='Дата',
        yaxis_title='Количество собранных рейсов в день',
        template='plotly_white'
    )
    fig.update_traces(
        hovertemplate='Date: %{x|%Y-%m-%d}<br>Количество: %{y:.2f}<extra></extra>'
    )
    fig.show()

In [112]:
make_flights_count_plot()